In [1]:
import numpy as np
import tensorflow as tf

In [9]:
# 检查是否启用 Eager Execution（默认已启用）
assert tf.executing_eagerly(), "Eager Execution should be enabled"

# -----------------------------
# 定义初始化权重函数（TF2.x 写法）
# -----------------------------
def init_weights(shape):
    return tf.Variable(tf.random.normal(shape, stddev=0.01))

# -----------------------------
# 定义神经网络结构（TF2.x 写法）
# -----------------------------
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # 输入层 dropout
    X = tf.nn.dropout(X, rate=1 - p_keep_input)  # keep -> drop: rate = 1 - keep

    # 隐藏层1
    h = tf.matmul(X, w_h)
    h = tf.nn.relu(h)

    # 隐藏层2 + dropout
    h = tf.nn.dropout(h, rate=1 - p_keep_hidden)
    h2 = tf.matmul(h, w_h2)
    h2 = tf.nn.relu(h2)

    # 输出层
    h2 = tf.nn.dropout(h2, rate=1 - p_keep_hidden)
    out = tf.matmul(h2, w_o)

    return out

In [ ]:
# -----------------------------
# 数据准备
# -----------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# one-hot 编码
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# -----------------------------
# 初始化模型参数
# -----------------------------
w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

# 学习率 & dropout 率
learning_rate = 0.001
p_keep_input_val = 0.8
p_keep_hidden_val = 0.5

# 优化器
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# -----------------------------
# 训练循环
# -----------------------------
@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, w_h, w_h2, w_o, p_keep_input_val, p_keep_hidden_val)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_batch))
    grads = tape.gradient(loss, [w_h, w_h2, w_o])
    optimizer.apply_gradients(zip(grads, [w_h, w_h2, w_o]))
    return loss

# 测试时不需要 dropout
def predict(x):
    return model(x, w_h, w_h2, w_o, 1.0, 1.0)

# 开始训练
for epoch in range(100):
    print(f"Epoch {epoch+1}")
    for start in range(0, len(x_train), 128):
        end = start + 128
        x_batch = x_train[start:end]
        y_batch = y_train[start:end]

        loss = train_step(x_batch, y_batch)

    # 每个 epoch 后评估测试准确率
    logits = predict(x_test)
    predictions = tf.argmax(logits, axis=1)
    true_labels = tf.argmax(y_test, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, true_labels), tf.float32))
    print(f"Test Accuracy: {accuracy.numpy():.4f}")

Epoch 1
Test Accuracy: 0.9437
Epoch 2
Test Accuracy: 0.9638
Epoch 3
Test Accuracy: 0.9710
Epoch 4
Test Accuracy: 0.9742
Epoch 5
Test Accuracy: 0.9769
Epoch 6
Test Accuracy: 0.9791
Epoch 7
Test Accuracy: 0.9800
Epoch 8
Test Accuracy: 0.9776
Epoch 9
Test Accuracy: 0.9806
Epoch 10
Test Accuracy: 0.9803
Epoch 11
Test Accuracy: 0.9817
Epoch 12
Test Accuracy: 0.9828
Epoch 13
Test Accuracy: 0.9823
Epoch 14
Test Accuracy: 0.9837
Epoch 15
Test Accuracy: 0.9832
Epoch 16
Test Accuracy: 0.9843
Epoch 17
Test Accuracy: 0.9832
Epoch 18
Test Accuracy: 0.9845
Epoch 19
Test Accuracy: 0.9843
Epoch 20
Test Accuracy: 0.9824
Epoch 21
Test Accuracy: 0.9839
Epoch 22
Test Accuracy: 0.9847
Epoch 23
Test Accuracy: 0.9845
Epoch 24
Test Accuracy: 0.9848
Epoch 25
Test Accuracy: 0.9852
Epoch 26
Test Accuracy: 0.9848
Epoch 27
Test Accuracy: 0.9852
Epoch 28
Test Accuracy: 0.9857
Epoch 29
Test Accuracy: 0.9852
Epoch 30
Test Accuracy: 0.9855
Epoch 31
Test Accuracy: 0.9858
Epoch 32
Test Accuracy: 0.9864
Epoch 33
Test Acc